## Heaps

In [4]:
class Heap(object):
    def __init__(self):
        self.heaplist = []
        
    def __len__(self):
        return len(self.heaplist)
    
    def __getitem__(self, idx):
        return self.heaplist[idx]
    
    def add(self, item):
        self.heaplist.append(item)
        self._heapify_up(len(self.heaplist) - 1)
    
    def _heapify_up(self, idx):
        parent = idx//2
        if self.heaplist[idx] < self.heaplist[parent]:
            self.heaplist[idx], self.heaplist[parent] = self.heaplist[parent], self.heaplist[idx]
            self._heapify_up(parent)
        
    def _get_min_child(self, idx):
        if ((idx * 2) + 2) > len(self.heaplist) - 1:
            return (idx * 2) + 1
        else:
            if self.heaplist[(idx * 2) + 1] < self.heaplist[(idx * 2) + 2]:
                return (idx * 2) + 1
            return (idx * 2) + 2
    
    def _heapify_down(self, idx):
        min_child = self._get_min_child(idx)
        if self.heaplist[idx] > self.heaplist[min_child]:
            self.heaplist[idx], self.heaplist[min_child] = self.heaplist[min_child], self.heaplist[idx]
            if not min_child*2 + 1 > len(self.heaplist) - 1:
                self._heapify_down(min_child)
    
    def del_min(self):
        last_element = self.heaplist.pop()
        self.heaplist[0] = last_element
        self._heapify_down(0)
        

In [5]:
def _is_min_heap(L, i):
    l, r = 2 * i + 1, 2 * i + 2

    if r < len(L): # has left and right children
        if L[l] < L[i] or L[r] < L[i]: # heap property is violated
            return False

        # check both children trees
        return _is_min_heap(L, l) and _is_min_heap(L, r)
    elif l < len(L): # only has left children
        if L[l] < L[i]: # heap property is violated
            return False

        # check left children tree
        return _is_min_heap(L, l)
    else: # has no children
        return True

print(_is_min_heap([-1, 1, 3, 5, 7], 0))
print(_is_min_heap([1, 5, 3, 7], 0))

True
True


In [6]:
a = Heap()
a.add(3)
print(_is_min_heap(a, 0))
a.add(5)
print(_is_min_heap(a, 0))
a.add(7)
print(_is_min_heap(a, 0))
a.add(1)
print(_is_min_heap(a, 0))
a.add(-1)
print(_is_min_heap(a, 0))
print(a.heaplist)
a.del_min()
print(_is_min_heap(a, 0))
print(a.heaplist)
a.del_min()
print(_is_min_heap(a, 0))
print(a.heaplist)
a.del_min()
print(_is_min_heap(a, 0))
print(a.heaplist)

True
True
True
True
True
[-1, 1, 3, 5, 7]
True
[1, 5, 3, 7]
True
[3, 5, 7]
True
[5, 7]


#### Merge k sorted lists
* Create a min heap of size k. Initialise with first element of each array
* Repeat while min heap is not empty
    * Pop min and add an element from the same array as the popped element
    * Add element to results
    * If there are no more elements from that array, do nothing

In [1]:
 def merge_sorted_arrays(sorted_arrays):
 
    min_heap = []
    # Builds a list of iterators for each array in sorted_arrays.
    sorted_arrays_iters = [iter(x) for x in sorted_arrays]

    # Puts first element from each iterator in min_heap.
    for i, it in enumerate(sorted_arrays_iters):
        first_element = next(it, None)
        if first_element is not None:
            heapq.heappush(min_heap, (first_element, i))

    result = []
    while min_heap:
        smallest_entry, smallest_array_i = heapq.heappop(min_heap)
        smallest_array_iter = sorted_arrays_iters[smallest_array_i]
        result.append(smallest_entry)
        next_element = next(smallest_array_iter, None)
        if next_element is not None:
            heapq.heappush(min_heap, (next_element, smallest_array_i))
    return result

#### Implement stack using heap
* We care about the most recently inserted element
    * This element should always be at the top
    * So attach a timestamp that can be used as the comparator
    * Use a max heap that captures the element with highest timestamp

#### Sort an increasing decreasing array
* Reverse the decreasing pieces of the array
* Now we have k sorted arrays that need merging
    * Use a heap of size k
    * get elements form array where it was popped from and add to heap
    * if no new elements exists do nothing

In [2]:
def sort_k_increasing_decreasing_array(A):
    # Decomposes A into a set of sorted arrays.
    sorted_subarrays = []
    INCREASING, DECREASING = range(2)
    subarray_type = INCREASING
    start_idx = 0
    for i in range(1, len(A) + 1):
        if (
            i == len(A) # A is ended. Adds the last subarray.
            or (A[i - 1] < A[i] and subarray_type == DECREASING)
            or (A[i - 1] >= A[i] and subarray_type == INCREASING)
        ):
            sorted_subarrays.append(A[start_idx:i] if subarray_type ==
                                 INCREASING else A[i - 1:start_idx - 1:-1])
            start_idx = i
            subarray_type = (
                DECREASING if subarray_type == INCREASING else INCREASING
            )
    return merge_sorted_arrays(sorted_subarrays)


#### Sort an almost sorted array
* Each number is atmost k distance away
* After the first k + 1 numbers have been read, the smallest number in the group is the smallest number
    * maintain a rolling heap of size k + 1
    * keep popping and adding to result and move to end like a sliding window

In [3]:
def sort_approximately_sorted_array(sequence, k):

    min_heap = []
    # Adds the first k elements into min_heap. Stop if there are fewer than k
    # elements.
    for x in itertools.islice(sequence, k):
        heapq.heappush(min_heap, x)

    result = []
    # For every new element, add it to min_heap and extract the smallest.
    for x in sequence:
        smallest = heapq.heappushpop(min_heap, x)
        result.append(smallest)

    # sequence is exhausted, iteratively extracts the remaining elements.
    while min_heap:
        smallest = heapq.heappop(min_heap)
        result.append(smallest)

    return result

#### K closest planets to earth
* Each planet is a certain distance away
    * maintain max heap of size k
    * Keep adding elements, evict elements that are greater than the result
    * maintain a rolling heap of size k + 1